In [2]:
import time
import pandas as pd
import requests
import json
from typing import List

In [3]:
# %pip install polars
import polars as pl

In [4]:
from extract_data import get_URLs,\
get_incident_ids, generate_canadian_avalanche_data

Time to taken for extract_data.py to run: 0.0s.


In [5]:
url = "https://incidents.avalanche.ca/public/incidents/?format=json"

In [6]:
URLS = get_URLs(url)

In [7]:
ids = get_incident_ids(URLS)

In [8]:
generate_canadian_avalanche_data(ids)

'Data was saved.'

In [ ]:
def generate_canadian_avalanche_data(incident_ids: List[str]) -> str:
    df = pd.DataFrame()
    for id in incident_ids:
        data = requests.get(
            "https://incidents.avalanche.ca/public/incidents/{}/?format=json".format(id)
        ).json()
        new_df = pd.json_normalize(data)
        df = pd.concat([df, new_df])
    df.to_csv("./data/can_avs_raw.csv")

    return "Data was saved."